# 10 minutes to... the differences between "elevation band" and "centerline" flowlines

In version 1.4, OGGM introduced a new way to compute flowlines: the so-called "elevation-band flowlines" (after [Huss & Farinotti, 2012](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2012JF002523)). These eleveation bands complement the already available "multiple centerlines" glacier representation. This notebook allows you to compare the two representations.

In [ ]:
from oggm import cfg, utils, workflow, graphics, tasks
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
cfg.initialize(logging_level='WARNING')

In [ ]:
# Pick the glacier you want! We try Hintereisferner
rgi_ids = ['RGI60-11.00897']

## Get ready

In order to open the same glacier on two different glacier directories, we trick: we set a new working directory for each case! This trick is not recommended for real runs: if you have a use case for such a workflow (the same glacier with different data, please [get in touch with us](https://github.com/OGGM/oggm/issues)).

In [ ]:
# Geometrical centerline
# Where to store the data 
cfg.PATHS['working_dir'] = utils.gettempdir(dirname='OGGM-centerlines', reset=True)

# We start from prepro level 3 with all data ready - note the url here
base_url = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L3-L5_files/2023.1/centerlines/W5E5/'
gdirs = workflow.init_glacier_directories(rgi_ids, from_prepro_level=3, prepro_border=80, prepro_base_url=base_url)
gdir_cl = gdirs[0]
gdir_cl

In [ ]:
# Elevation band flowline
# New working directory
cfg.PATHS['working_dir'] = utils.gettempdir(dirname='OGGM-elevbands', reset=True)

# Note the new url
base_url = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L3-L5_files/2023.1/elev_bands/W5E5/'
gdirs = workflow.init_glacier_directories(rgi_ids, from_prepro_level=3, prepro_border=80, prepro_base_url=base_url)
gdir_eb = gdirs[0]
gdir_eb

## Some reading first 

we wrote a bit of information about the differences between these to. First, go to the [glacier flowlines](https://docs.oggm.org/en/stable/flowlines.html#glacier-flowlines) documentation where you can find detailed information about the two flowline types and also a [guideline when to use which flowline method](https://docs.oggm.org/en/stable/flowlines.html#pros-and-cons-of-both-methods).

The examples below illustrate these differences, without much text for now because of lack of time:

##  Glacier length and cross section

In [ ]:
fls_cl = gdir_cl.read_pickle('model_flowlines')
fls_eb = gdir_eb.read_pickle('model_flowlines')

In [ ]:
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 14), sharex=True, sharey=True)
graphics.plot_modeloutput_section(fls_cl, ax=ax1)
ax1.set_title('Geometrical centerline')
graphics.plot_modeloutput_section(fls_eb, ax=ax2)
ax2.set_title('Elevation band flowline');

**Note that the elevation band flowline length is shorter than the geometrical centerline!**

## Projections: very small differences in volume in most cases

Preparing the data is the same for both products. However, centerlines need to run with a different evolution model.

Elevation band flowlines are considerably faster to run.

In [ ]:
gdirs = [gdir_cl, gdir_eb]

In [ ]:
from oggm.shop import gcm_climate

# you can choose one of these 5 different GCMs:
# 'gfdl-esm4_r1i1p1f1', 'mpi-esm1-2-hr_r1i1p1f1', 'mri-esm2-0_r1i1p1f1' ("low sensitivity" models, within typical ranges from AR6)
# 'ipsl-cm6a-lr_r1i1p1f1', 'ukesm1-0-ll_r1i1p1f2' ("hotter" models, especially ukesm1-0-ll)
member = 'mri-esm2-0_r1i1p1f1' 

for ssp in ['ssp126', 'ssp370','ssp585']:
    # bias correct them
    workflow.execute_entity_task(gcm_climate.process_monthly_isimip_data, gdirs, 
                                 ssp = ssp,
                                 # gcm member -> you can choose another one
                                 member=member,
                                 # recognize the climate file for later
                                 output_filesuffix=f'_ISIMIP3b_{member}_{ssp}'
                                 );

In [ ]:
cfg.PARAMS['store_model_geometry'] = True  # add additional outputs for the maps below
cfg.PARAMS['store_fl_diagnostics'] = True
cfg.PARAMS['evolution_model'] = 'FluxBased'

workflow.execute_entity_task(tasks.run_from_climate_data, [gdir_cl],
                             output_filesuffix='_historical',  
                            );

for ssp in ['ssp126', 'ssp370', 'ssp585']:
    rid = f'_ISIMIP3b_{member}_{ssp}'

    workflow.execute_entity_task(tasks.run_from_climate_data, [gdir_cl],
                                 climate_filename='gcm_data',  # use gcm_data, not climate_historical
                                 climate_input_filesuffix=rid,  # use the chosen scenario
                                 init_model_filesuffix='_historical',  # this is important! Start from 2020 glacier
                                 output_filesuffix=rid,  # recognize the run for later
                                );

In [ ]:
cfg.PARAMS['store_model_geometry'] = True  # add additional outputs for the maps below
cfg.PARAMS['store_fl_diagnostics'] = True
cfg.PARAMS['evolution_model'] = 'SemiImplicit'

workflow.execute_entity_task(tasks.run_from_climate_data, [gdir_eb],
                             output_filesuffix='_historical', 
                            );

for ssp in ['ssp126', 'ssp370', 'ssp585']:
    rid = f'_ISIMIP3b_{member}_{ssp}'

    workflow.execute_entity_task(tasks.run_from_climate_data, [gdir_eb],
                                 climate_filename='gcm_data',  # use gcm_data, not climate_historical
                                 climate_input_filesuffix=rid,  # use the chosen scenario
                                 init_model_filesuffix='_historical',  # this is important! Start from 2020 glacier
                                 output_filesuffix=rid,  # recognize the run for later
                                );

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
# Pick some colors for the lines
color_dict={'ssp126':'blue', 'ssp370':'orange', 'ssp585':'red'}
for ssp in ['ssp126','ssp370', 'ssp585']:
    rid = f'_ISIMIP3b_{member}_{ssp}'
    with xr.open_dataset(gdir_eb.get_filepath('model_diagnostics', filesuffix=rid)) as ds:
        ds.volume_m3.plot(ax=ax1, label=ssp, c=color_dict[ssp]);
    ax1.set_title('Centerlines')
for ssp in ['ssp126','ssp370', 'ssp585']:
    rid = f'_ISIMIP3b_{member}_{ssp}'
    with xr.open_dataset(gdir_cl.get_filepath('model_diagnostics', filesuffix=rid)) as ds:
        ds.volume_m3.plot(ax=ax2, label=ssp, c=color_dict[ssp]);
    ax2.set_title('Elevation bands')
plt.legend();

As you can see, for this disappearing glacier, the differences between the representations are almost indiscernable. The differences can be a bit larger at times, for example in length:

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
# Pick some colors for the lines
color_dict={'ssp126':'blue', 'ssp370':'orange', 'ssp585':'red'}
for ssp in ['ssp126','ssp370', 'ssp585']:
    rid = f'_ISIMIP3b_{member}_{ssp}'
    with xr.open_dataset(gdir_eb.get_filepath('model_diagnostics', filesuffix=rid)) as ds:
        ds.length_m.plot(ax=ax1, label=ssp, c=color_dict[ssp]);
    ax1.set_title('Centerlines')
for ssp in ['ssp126','ssp370', 'ssp585']:
    rid = f'_ISIMIP3b_{member}_{ssp}'
    with xr.open_dataset(gdir_cl.get_filepath('model_diagnostics', filesuffix=rid)) as ds:
        ds.length_m.plot(ax=ax2, label=ssp, c=color_dict[ssp]);
    ax2.set_title('Elevation bands')
plt.legend();

## Graphical representation: centerlines win by short margin (for now)

In [ ]:
rid = f'_ISIMIP3b_{member}_ssp126'

Both models can be reprensented like this: 

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
with xr.open_dataset(gdir_cl.get_filepath('fl_diagnostics', filesuffix=rid), group='fl_2') as ds:
    (ds.bed_h + ds.sel(time=[2021, 2030, 2040, 2050]).thickness_m).plot(ax=ax1, hue='time')
    ds.bed_h.plot(ax=ax1, c='k')
with xr.open_dataset(gdir_eb.get_filepath('fl_diagnostics', filesuffix=rid), group='fl_0') as ds:
    (ds.bed_h + ds.sel(time=[2021, 2030, 2040, 2050]).thickness_m).plot(ax=ax2, hue='time')
    ds.bed_h.plot(ax=ax2, c='k')

However, only with centerlines you can plot this:

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(14, 6))
graphics.plot_modeloutput_map(gdir_cl, filesuffix=rid, modelyr=2021, ax=ax1, vmax=150)
graphics.plot_modeloutput_map(gdir_cl, filesuffix=rid, modelyr=2050, ax=ax2, vmax=150)
graphics.plot_modeloutput_map(gdir_cl, filesuffix=rid, modelyr=2100, ax=ax3, vmax=150)
plt.tight_layout();

## What's next?

- return to the [OGGM documentation](https://docs.oggm.org)
- back to the [table of contents](welcome.ipynb)